In [385]:
import os

import bk.load
import bk.compute
import bk.plot
import bk.signal

import numpy as np
import pandas as pd

import scipy.signal
import scipy.stats as ss
import scipy.io

import matplotlib.pyplot as plt
import matplotlib as mpl
import neuroseries as nts
import basefunction.vBaseFunctions3
from tqdm import tqdm
import itertools

In [396]:
def save_fig(folder_path,name):
    import os
    old_path = os.getcwd()
    os.chdir(folder_path)
    plt.savefig(name)
    os.chdir(old_path)

def load_ripples():
    ripples_ = scipy.io.loadmat(f'{bk.load.session}-RippleFiring.mat')['ripples']['allsws'][0][0]
#     ripples_ = pd.DataFrame(data = ripples,columns=['start','peak','stop'])
    
    columns = ['start','peak','stop']
    
    ripples = {}
    for i,c in zip(range(ripples_.shape[1]),columns):
        ripples.update({c:nts.Ts(ripples_[:,i],time_units='s')})
    return ripples

def poissonTest(baseRate,counts,time):
    eps = np.spacing(1)
    
    lam = baseRate*time
    
    pInc = 1 - ss.poisson.cdf(counts,lam)
    pDec = ss.poisson.cdf(counts,lam)
    surprise = np.log((pDec+eps)/(pInc + eps))
    return pInc,pDec,surprise

def psth(neurons,stimulus,binSize,win):
    
    if isinstance(neurons,nts.time_series.Tsd): 
        print('Converted')
        neurons = np.array()
    winLen = int((win[1] - win[0])/binSize)
    window = np.arange(winLen,dtype = int)-int(winLen/2)
    stim_bin = (stimulus/binSize).astype('int')
    t,binned = bk.compute.binSpikes(neurons,binSize)
    print(binned.shape)
    print(stim_bin[-1])
    psth = np.empty((stimulus.size,len(neurons),winLen))
    
    for i,t in tqdm(enumerate(stim_bin)):
        psth[i] = binned[:,t+window]
        
    psth = np.mean(psth,0).T
    t = window*binSize
    return t,psth

In [397]:
bk.load.current_session()
neurons,metadata = bk.load.loadSpikeData(bk.load.path)

Rat : 8 on day : 6
Working with session Rat08-20130713 @ Z:\Rat08\Rat08-20130713
Data already saved in Numpy format, loading them from here:
Rat08-20130713-neurons.npy
Rat08-20130713-metadata.npy


In [398]:
t,p = psth(np.array(neurons[0:1]),neurons[0].as_units('s').index.values,0.001,[-0.1,0.1])

15783it [00:00, 159551.85it/s]

(1, 22676917)
22676917


IndexError: index 22676917 is out of bounds for axis 1 with size 22676917

In [351]:
a = np.array([1,2,3])

In [360]:
a[3]

IndexError: index 3 is out of bounds for axis 0 with size 3

In [311]:
a

array([[nan, nan, nan, ..., nan, nan, nan]], dtype=object)

In [228]:
for i in a: print(i)

1


In [198]:
p.shape

(200, 1)

In [258]:
plt.bar(t,p[:,0],t[1]-t[2])

<BarContainer object of 200 artists>

In [172]:
def ripple_psth(path):
    
    win = [-0.5,0.5]
    binSize = 0.002
    
    bk.load.current_session(path)
    ripples = load_ripples()
    neurons,metadata = bk.load.loadSpikeData(bk.load.path)
    
    ripples_ = ripples['peak'].as_units('s').index.values
    t,psth_ = psth(neurons,ripples_,binSize,win)
    
    states = bk.load.states()
    states.update({'ripples':nts.IntervalSet(ripples['start'].times(),ripples['stop'].times())})
    states.update({'outside_ripples':states['sws'].set_diff(nts.IntervalSet(states['ripples'].start-200_000,states['ripples'].end+200_000,force_no_fix=True))})
    
    pInc = np.empty(len(neurons))
    pDec = np.empty(len(neurons))
    surprise = np.empty(len(neurons))
    for i,n in enumerate(neurons):
        baseRate = len(n.restrict(states['outside_ripples']))/states['outside_ripples'].tot_length(time_units = 's')
        count = len(n.restrict(states['ripples']))
        time = states['ripples'].tot_length(time_units = 's')
        
        pInc[i],pDec[i],surprise[i] = poissonTest(baseRate,count,time)
    
    stats = pd.DataFrame(np.array([pInc,pDec,surprise]).T,columns=['pInc','pDec','surprise'])
    
    metadata = pd.concat([metadata,stats],1) 
    
    return t,psth_,metadata

In [68]:
def ripple_psth_slow(path):
    nbins = 100
    
    
    bk.load.current_session(path)
    ripples = load_ripples()
    neurons,metadata = bk.load.loadSpikeData(bk.load.path)

    win = [-0.5,0.5]
    n_neurons = len(neurons)
    
    peth = np.empty((len(ripples['peak'].as_units('s').index),nbins,n_neurons))
    
    for i,ripple_time in tqdm(enumerate(ripples['peak'].as_units('s').index)):
        t,binned = bk.compute.binSpikes(neurons,nbins=nbins,start = ripple_time+win[0],stop = ripple_time+win[1])
        peth[i,:] = binned.T
    
    
    states = bk.load.states()
    states.update({'ripples':nts.IntervalSet(ripples['start'].times(),ripples['stop'].times())})
    states.update({'outside_ripples':states['sws'].set_diff(nts.IntervalSet(states['ripples'].start-200_000,states['ripples'].end+200_000,force_no_fix=True))})
    
    pInc = np.empty(len(neurons))
    pDec = np.empty(len(neurons))
    surprise = np.empty(len(neurons))
    for i,n in enumerate(neurons):
        baseRate = len(n.restrict(states['outside_ripples']))/states['outside_ripples'].tot_length(time_units = 's')
        count = len(n.restrict(states['ripples']))
        time = states['ripples'].tot_length(time_units = 's')
        
        pInc[i],pDec[i],surprise[i] = poissonTest(baseRate,count,time)
    
    stats = pd.DataFrame(np.array([pInc,pDec,surprise]).T,columns=['pInc','pDec','surprise'])
    
    
    return t,peth,metadata,stats

In [145]:
batch = bk.load.batch(ripple_psth)

100%|███████████████████████████████████████████████████████████████████████████████████████████| 61/61 [1:40:07<00:00, 98.49s/it]

Batch finished in 6007.606546163559
Some session were not processed correctly
['Rat08-20130708', 'Rat08-20130709', 'Rat08-20130712', 'Rat08-20130716', 'Rat08-20130718', 'Rat08-20130719', 'Rat08-20130720', 'Rat08-20130722', 'Rat09-20140324', 'Rat09-20140325', 'Rat09-20140326', 'Rat09-20140327', 'Rat09-20140328', 'Rat09-20140329', 'Rat09-20140331', 'Rat09-20140401', 'Rat09-20140402', 'Rat09-20140403', 'Rat09-20140404', 'Rat09-20140405', 'Rat09-20140407', 'Rat09-20140408', 'Rat09-20140409', 'Rat10-20140629', 'Rat10-20140704', 'Rat10-20140707', 'Rat10-20140708', 'Rat11-20150315', 'Rat11-20150316', 'Rat11-20150402', 'Rat11-20150403']
50.81967213114754 %


In [146]:
metaall = pd.concat([batch[s][2] for s in batch])
psthall = np.hstack([batch[s][1] for s in batch])

In [147]:
temps = batch['Rat08-20130710'][0]

In [155]:
%matplotlib qt

In [169]:
stru = np.unique(metaall.Region)
types = ['Pyr','Int']
# stru = ['Hpc','BLA']
for s,t in itertools.product(stru,types):
    mask = (metaall.pInc < 0.001) & (metaall.Region == s) & (metaall.Type == t)
    mask = list(mask)
    psth_stru_types_u = np.nanmean(psthall[:,np.where(mask)[0]],1)
    
    mask = (metaall.pDec < 0.001) & (metaall.Region == s) & (metaall.Type == t)
    mask = list(mask)
    psth_stru_types_d = np.nanmean(psthall[:,np.where(mask)[0]],1)
    
    plt.figure()
    
    plt.suptitle(f'{s} and {t}')
    
    plt.subplot(1,2,1)
    plt.title('Up-Mod')
    plt.bar(temps,psth_stru_types_u,0.002,alpha = 0.5)
    plt.axvline(0,color = 'r',alpha = 0.5)
    
    plt.xlim(-0.5,0.5)
    
    plt.subplot(1,2,2)
    plt.title('Down-Mod')
    plt.bar(temps,psth_stru_types_d,0.002,alpha = 0.5)
    plt.axvline(0,color = 'r',alpha = 0.5)
    plt.xlim(-0.5,0.5)

    
    plt.tight_layout()
    save_fig('P:/Data/Figures/',f'{s}-{t}-0.002-full.svg')
    save_fig('P:/Data/Figures/',f'{s}-{t}-0.002-full.png')
    

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: Mean of empty slice
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: Mean of empty slice
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: Mean of empty slice
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: Mean of empty slice
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: Mean of empty slice
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: Mean of empty slice
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: 

In [166]:
os.getcwd()

'P:\\Data\\Figures'